In [14]:
import pandas as pd

/Users/ncg/Desktop/GoogleDrive/venv/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# Basics

In [2]:
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [23]:
# this enables you to read from the spreadsheets
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

In [24]:
# you can find your sheet ID by going to the url of your sheet and
#    copying the extension between spreadsheets/d/ and /edit
SHEET_ID = '1XdVQrG6LnBx9JpN8zMPLpiUHhccd2SCcuYSDDzjUF64'

In [25]:
# this is just asking what subset of the data to include.
# if you want to include everything, a quick trick is to just make a 
# pivot table after you highlight your data and copy the data range
SHEET_RANGE = 'Sheet1!A1:B2'

#### USER NOTE: I made the path to my credentials include a '..' because this notebook is located in a subdirectory off the root directory. If you have your credentials.json in the same folder as the notebook you're using to follow along, the path will simply be 'credentials.json' without the '..'

* This will create a file called token.pickle in your working directory that you can use the code below to access your Google Sheets API at any point.

In [26]:
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            '../credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

In [27]:
service = build('sheets', 'v4', credentials=creds)

In [28]:
# Call the Sheets API
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SHEET_ID,
                            range=SHEET_RANGE).execute()
values = result.get('values', [])

#### USER NOTE: Here's a useful function to create a df with first row headers from the values.

In [29]:
def create_df_with_first_row_headers(sheets_values):
    df = pd.DataFrame(sheets_values)
    
    # replaces first row of dataframe with the headers b/c it's not automatic
    headers = df.iloc[0]
    df = df[1:]
    df.columns = headers
    
    return df

In [30]:
df = create_df_with_first_row_headers(values)

In [31]:
df.head()

,Name,Data
1,Hello World,Noah


# Writing to Spreadsheet

#### USER NOTE: Below, I've created a couple 'wrapper' functions that cover some basic use of the Sheets API. 
* This is because the initial API is a bit specific & complicated for new users and it provides too many ways to use it that a beginning or brief user won't need to cover.
* What I mean by specific is that you need to enter in the specific range of the cells you want to update. The wrapper functions simply add a new line to the end of the range in your file.

In [32]:
values = [
    [
        'Hello Earth',
        'Rafael'
    ]
]

body = {'values': values}

In [33]:
# valueInputOption is either RAW or USER_ENTERED
result = service.spreadsheets().values().update(
    spreadsheetId=SHEET_ID, range='Sheet1!A3:B3',
    valueInputOption='USER_ENTERED', body=body).execute()

HttpError: <HttpError 403 when requesting https://sheets.googleapis.com/v4/spreadsheets/1XdVQrG6LnBx9JpN8zMPLpiUHhccd2SCcuYSDDzjUF64/values/Sheet1%21A3%3AB3?valueInputOption=USER_ENTERED&alt=json returned "Request had insufficient authentication scopes.">